In [26]:
import os

In [27]:
%pwd

'd:\\Machine Learning\\Wine-Classification'

In [28]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class ModelTrainingConfig:
      root_dir: Path
      train_data: Path
      test_data: Path
      model: str
      alpha: float
      l1_ration: float
      target_column: str

In [29]:
from src.wineModel.utils import *
from src.wineModel.constants import *


class ConfigurationManager:
      def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
      ):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directory([self.config.artifacts_root])

      
      def get_model_training_config(self) -> ModelTrainingConfig:
            config = self.config.model_trainer
            params = self.params.ElasticNet
            schema = self.schema.TARGET_COLUMN

            create_directory([config.root_dir])

            model_train_config = ModelTrainingConfig(
                  root_dir = config.root_dir,
                  train_data = config.train_data,
                  test_data = config.test_data,
                  model = config.model,
                  alpha = params.alpha,
                  l1_ration = params.l1_ratio,
                  target_column = schema.name
            )
            return model_train_config

In [30]:
import pandas as pd
import os
from src.wineModel import logger
from sklearn.linear_model import ElasticNet
import joblib


class ModelTrainig:
      def __init__(self, config: ModelTrainingConfig):
            self.config = config
      
      def model_train(self):
            train_data = pd.read_csv(self.config.train_data)
            test_data = pd.read_csv(self.config.test_data)

            x_train = train_data.drop(columns='quality')
            x_val = test_data.drop(columns='quality')
            y_train = train_data[['quality']]
            y_val = test_data[['quality']]

            model = ElasticNet(
                  alpha=self.config.alpha,
                  l1_ratio=self.config.l1_ration,
                  random_state=42
            )

            model.fit(x_train, y_train)

            joblib.dump(model, os.path.join(self.config.root_dir, self.config.model))
            logger.info("Model saved successfully")

In [31]:
try:
      config = ConfigurationManager()
      model_training_config = config.get_model_training_config()
      model_train = ModelTrainig(config=model_training_config)
      model_train.model_train()
except Exception as e:
      raise e

[2026-02-12 19:39:53,417: INFO: __init__: yaml file: config\config.yaml is loaded successfully]
[2026-02-12 19:39:53,420: INFO: __init__: yaml file: params.yaml is loaded successfully]
[2026-02-12 19:39:53,424: INFO: __init__: yaml file: schema.yaml is loaded successfully]
[2026-02-12 19:39:53,426: INFO: __init__: Directory is created at: artifacts]
[2026-02-12 19:39:53,428: INFO: __init__: Directory is created at: artifacts/model_trainer]
[2026-02-12 19:39:53,472: INFO: 2383909173: Model saved successfully]
